In [1]:

import cv2
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
import numpy as np
import shutil
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2,l1
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.multiclass import unique_labels
from keras.optimizers import RMSprop, SGD, Adam
# from keras.regularizers import regularizers
from numpy import load
from  keras.preprocessing.image import DirectoryIterator

from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


### Importing dataset

In [3]:
train_path_alpha="/home/kashraf/felix_hd/topomap+csv_entire/topomap/alpha/train/"
train_path_beta="/home/kashraf/felix_hd/topomap+csv_entire/topomap/beta/train/"
train_path_theta= "/home/kashraf/felix_hd/topomap+csv_entire/topomap/theta/train/"

test_path_alpha="/home/kashraf/felix_hd/topomap+csv_entire/topomap/alpha/test/"
test_path_theta="/home/kashraf/felix_hd/topomap+csv_entire/topomap/theta/test/"
test_path_beta= "/home/kashraf/felix_hd/topomap+csv_entire/topomap/beta/test/"

### Keras data generator

In [5]:
train_datagen=ImageDataGenerator( rescale=1./255,
      rotation_range=0,
      width_shift_range=0,
      height_shift_range=0,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen=ImageDataGenerator( rescale=1./255)

train_data=[train_datagen.flow_from_directory(train_path_theta,batch_size=32,shuffle=False),
         train_datagen.flow_from_directory(train_path_alpha,batch_size=32,shuffle=False),
         train_datagen.flow_from_directory(train_path_beta,batch_size=32,shuffle=False)]

test_data=[test_datagen.flow_from_directory(test_path_theta,shuffle=False),
          test_datagen.flow_from_directory(test_path_alpha,shuffle=False),
          test_datagen.flow_from_directory(test_path_beta,shuffle=False)]


Found 30240 images belonging to 4 classes.
Found 30240 images belonging to 4 classes.
Found 30240 images belonging to 4 classes.
Found 13760 images belonging to 4 classes.
Found 13760 images belonging to 4 classes.
Found 13760 images belonging to 4 classes.


### Lett's build our essemble model

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [11]:
# model = Sequential()
num_classes=4
nets = 3
img_rows=256
img_cols=256
model = [0] *nets

for j in range(nets):
    model[j]= Sequential()
 # First CONV-ReLU Layer
    model[j].add(Conv2D(32, (3, 3), padding = 'same', input_shape = (img_rows, img_cols,3)))
    model[j].add(BatchNormalization())
    model[j].add(Activation('relu'))
    model[j].add(MaxPooling2D(pool_size=(2, 2)))

    # Second CONV-ReLU Layer
    model[j].add(Conv2D(64, (3, 3), padding = "same"))
    model[j].add(BatchNormalization())
    model[j].add(Activation('relu'))
    model[j].add(MaxPooling2D(pool_size=(2, 2)))

    model[j].add(Conv2D(96, (3, 3), padding="same"))
    model[j].add(BatchNormalization())
    model[j].add(Activation('relu'))
    model[j].add(MaxPooling2D(pool_size=(2, 2)))

    model[j].add(Conv2D(128, (3, 3), padding="same"))
    model[j].add(BatchNormalization())
    model[j].add(Activation('relu'))
    model[j].add(MaxPooling2D(pool_size=(2, 2)))

    model[j].add(Conv2D(256, (3, 3), padding="same"))
    model[j].add(BatchNormalization())
    model[j].add(Activation('relu'))
    model[j].add(MaxPooling2D(pool_size=(2, 2)))

    model[j].add(Conv2D(256, (3, 3), padding="same"))
    model[j].add(BatchNormalization())
    model[j].add(Activation('relu'))
    model[j].add(MaxPooling2D(pool_size=(2, 2)))
    model[j].add(Flatten())

    model[j].add(Dense(1024))
    model[j].add(Activation('relu'))
    model[j].add(Dropout(0.4))


    # Final Dense Layer
    model[j].add(Dense(num_classes,activation='softmax',name='mbm'))
    model[j].compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.0001),
              metrics = ['accuracy'])

print("All models built and compiled")
print(model[0].summary())

All models built and compiled
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
activation_2 (Activation

## Now Let's train our models

In [ ]:
# Training Parameters
# batch_size=16
epochs=60
checkpoint = [ModelCheckpoint("/home/kashraf/Felix_work2/2021_research/essemble_models/essembel_sub_theta--2.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1),
               ModelCheckpoint("/home/kashraf/Felix_work2/2021_research/essemble_models/essembel_sub_alpha--2.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1),
               ModelCheckpoint("/home/kashraf/Felix_work2/2021_research/essemble_models/essembel_sub_beta--2.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)]


reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 4,
                              verbose = 1,
                              min_delta = 0.000001)

# we put our call backs into a callback list
callbacks =[ [ checkpoint[0],reduce_lr],[ checkpoint[1],reduce_lr],[ checkpoint[2],reduce_lr]]


history =[0]*nets 
for i in range(nets):
    history[i] = model[i].fit_generator(train_data[i], 
                                         epochs = epochs,validation_data =test_data[i],
                                         callbacks=callbacks[i], verbose=1)
    
# model.save("/home/kashraf/Felix_work2/modelsDL/stack_AlexNet_1_Epoch.h5")

# Evaluate the performance of our trained model
score_alpha= model[0].evaluate(x_test[0], y_test[0], verbose=1)
score_theta= model[1].evaluate(x_test[1], y_test[1], verbose=1)
score_beta= model[2].evaluate(x_test[2], y_test[2], verbose=1)

print('Test loss theta:', score_theta[0])
print('Test accuracy theta: ', score_theta[1])

print('Test loss alpha:', score_alpha[0])
print('Test accuracy alpha :', score_alpha[1])

print('Test loss beta:', score_beta[0])
print('Test accuracy beta:', score_beta[1])

Epoch 1/60
945/945 [==============================] - 1562s 2s/step - loss: 0.9625 - accuracy: 0.6520 - val_loss: 0.2101 - val_accuracy: 0.9027

Epoch 00001: val_loss improved from inf to 0.21008, saving model to /home/kashraf/Felix_work2/2021_research/essemble_models/essembel_sub_theta--2.h5
Epoch 2/60
945/945 [==============================] - 1559s 2s/step - loss: 0.1869 - accuracy: 0.9310 - val_loss: 0.0210 - val_accuracy: 0.9217

Epoch 00002: val_loss improved from 0.21008 to 0.02098, saving model to /home/kashraf/Felix_work2/2021_research/essemble_models/essembel_sub_theta--2.h5
Epoch 3/60
945/945 [==============================] - 1558s 2s/step - loss: 0.0713 - accuracy: 0.9762 - val_loss: 0.0043 - val_accuracy: 0.9698

Epoch 00003: val_loss improved from 0.02098 to 0.00429, saving model to /home/kashraf/Felix_work2/2021_research/essemble_models/essembel_sub_theta--2.h5
Epoch 4/60
945/945 [==============================] - 1558s 2s/step - loss: 0.0478 - accuracy: 0.9847 - val_lo

### Put our data in the right

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=4)

x_train = x_train.astype('float16')
x_test = x_test.astype('float16')


In [ ]:
import keras
y_train = keras.utils.to_categorical(y_train, num_classes=4)
y_test = keras.utils.to_categorical(y_test, num_classes=4)

### Loading our pre_trained Bany's model

In [ ]:
model = Sequential()
num_classes=4

# First CONV-ReLU Layer
model.add(Conv2D(32, (7, 7), padding = 'same', input_shape = (img_rows, img_cols,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second CONV-ReLU Layer
model.add(Conv2D(64, (5, 5), padding = "same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding="same",kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding="same",kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding="same",kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))


# Final Dense Layer
model.add(Dense(num_classes,activation='softmax',name='mbm'))

print(model.summary())

## Compiling the model


In [ ]:
# Training Parameters
checkpoint = ModelCheckpoint("/home/kashraf/Felix_work2/modelsDL/theta_topomap-weights--2.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 4,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 4,
                              verbose = 1,
                              min_delta = 0.000001)

# we put our call backs into a callback list
callbacks = [ checkpoint,reduce_lr]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.0001),
              metrics = ['accuracy'])
batch_size=16
epochs=100

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          callbacks = callbacks,
          validation_data=(x_test, y_test),
          shuffle=True)

# model.save("/home/kashraf/Felix_work2/modelsDL/stack_AlexNet_1_Epoch.h5")

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
images.shape

### Saving our model history


In [ ]:
import pickle 

pickle_out = open("beta_history.pickle","wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()
# pickle_in = open("alphaAlexnet_history.pickle","rb")
# saved_history = pickle.load(pickle_in)
# print(saved_history)

### Model performance visualization

In [ ]:
# Plotting our loss charts
import matplotlib.pyplot as plt

history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Plotting our accuracy charts
import matplotlib.pyplot as plt

history_dict = history.history

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

### Confusion matrix

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

y_pred = model.predict_classes(x_test)

print(classification_report(np.argmax(y_test,axis=1), y_pred))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

### Saliency Visualizations
To visualize activation over final dense layer outputs, we need to switch the softmax activation out for linear since gradient of output node will depend on all the other node activations. Doing this in keras is tricky, so we provide utils.apply_modifications to modify network parameters and rebuild the graph.

If this swapping is not done, the results might be suboptimal. We will start by swapping out 'softmax' for 'linear' and compare what happens if we dont do this at the end.

Lets pick an input over which we want to show the attention

In [ ]:
from vis.visualization import visualize_saliency
from vis.utils import utils
from keras import activations
from matplotlib import pyplot as plt
%matplotlib inline

class_idx = 0
indices = np.where(y_test[:, class_idx] == 1.)[0]

# pick some random input from here.
idx = indices[0]

# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx =-1

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

grads = visualize_saliency(model, layer_idx, filter_indices=class_idx, seed_input=x_test[idx])
# Plot with 'jet' colormap to visualize as a heatmap.
plt.imshow(grads, cmap='RdBu_r')


To used guided saliency, we need to set backprop_modifier='guided'. For rectified saliency or deconv saliency, use backprop_modifier='relu'. Lets try these options quickly and see how they compare to vanilla saliency.

In [ ]:
for modifier in ['guided', 'relu']:
    grads = visualize_saliency(model, layer_idx, filter_indices=class_idx,
                               seed_input=x_test[idx], backprop_modifier=modifier)
    plt.figure()
    plt.title(modifier)
    plt.imshow(grads, cmap='RdBu_r')

In [ ]:
# This corresponds to the Dense linear layer.
for class_idx in np.arange(4):    
    indices = np.where(y_test[:, class_idx] == 1.)[0]
    idx = indices[0]
    
    plt.figure(figsize=(25,15))
    f, ax = plt.subplots(1, 4)
   
    ax[0].imshow(x_test[idx][..., 0])
    
    for i, modifier in enumerate([None, 'guided', 'relu']):
        grads = visualize_saliency(model, layer_idx, filter_indices=class_idx, 
                                   seed_input=x_test[idx], backprop_modifier=modifier)
        if modifier is None:
            modifier = 'vanilla'
        ax[i+1].set_title(modifier)    
        ax[i+1].imshow(grads, cmap='jet')